## Mount Google Drive

In [ ]:
from google.colab import drive

# Make sure to unmount drive at mount point
drive.flush_and_unmount()
drive.mount('/content/drive')

# Research Start

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pytz
import pandas as pd

from datetime import datetime
from pathlib import Path
from scipy import stats

%matplotlib inline

### Creating IDX Equities Time Series

In [2]:
csv_combined = Path.home() / Path('Documents/data-ab/idx_exported_combined.csv') # Windows and Mac Path
# csv_combined = Path.home() / Path('documents/amibroker/idx_exported_combined.csv') # Linux Path
# csv_combined = Path('/mnt/c/Users/nikki/Documents/data-ab/idx_exported_combined.csv') # WSL Path

# idx_combined = pd.read_csv(csv_combined, parse_dates={'Date' : [1]})

In [2]:
csv_loc = Path.home() / Path('Documents/data-ab/idx_exported_csv') # Windows and Mac Path
# csv_loc = Path.home() / Path('documents/amibroker/idx_exported_csv') # Linux Path
# csv_loc = Path(r'/content/drive/Shared drives/algo-clenow/idx_exported_csv') # Google Colab Path
# csv_loc = Path('/mnt/c/Users/nikki/Documents/data-ab/idx_exported_csv') # WSL Path

# IDX stocks' ticker always have 4 characters
files = list(csv_loc.glob('????.csv'))

In [3]:
"""
Create a dictionary where the key is the ticker
and the value is a pandas dataframe of the OHLC time series
"""
data_idx = {}
for file in files:
    data_idx[file.stem] = pd.read_csv(file,
                                      index_col='Date',
                                      parse_dates=['Date'])
    # data_idx[file.stem] = data_idx[file.stem].tz_localize(tz='Asia/Jakarta')

In [4]:
data_idx['ASII'].tail()

,Open,High,Low,Close,Volume
Date,,,,,
2021-07-21,4780.0,4930.0,4780.0,4870.0,41716000
2021-07-22,4930.0,5025.0,4900.0,4980.0,36287300
2021-07-23,5000.0,5000.0,4910.0,4950.0,26847700
2021-07-26,4950.0,4970.0,4770.0,4770.0,43373400
2021-07-27,4770.0,4830.0,4730.0,4760.0,42635500


### Stocks on the Move


In [5]:
def momentum_score(ts):
    """
    Input:  Price time series.
    Output: Annualized exponential regression slope, 
            multiplied by the R2
    """
    # Make a list of consecutive numbers
    x = np.arange(len(ts)) 
    # Get logs
    log_ts = np.log(ts) 
    # Calculate regression values
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, log_ts)
    # Annualize percent
    annualized_slope = (np.power(np.exp(slope), 252) - 1) * 100
    #Adjust for fitness
    score = annualized_slope * (r_value ** 2)
    return score

In [6]:
def volatility(ts, period=24):
    """
    Input:  Price time series, Look back period
    Output: Standard deviation of the percent change
    """
    return ts.pct_change().rolling(period).std().iloc[-1]

In [7]:
# How many (series) candles back for momentum calculation?
momentum_window = 96

# Create an empty DataFrame to store score
momentum_table = pd.DataFrame(columns=['ticker', 'score', 'vola', 'inv_vola'])

# Eliminated stocks list
eliminated_stocks = pd.DataFrame(columns=['ticker', 'score', 'vola', 'reason'])

# How many (series) candles+1 back for std dev calculation?
vola_window = 24

# How many (series) candles back for EWMA calculation?
ma_period_fast = 32
ma_period_slow = 128

# Loop the dictionary and calculate the momentum_score, then append it to pandas
for ticker, timeseries in data_idx.items():
    momentum_series = timeseries['Close'].iloc[-momentum_window:]
    score = momentum_score(momentum_series)
    vola_series = timeseries['Close']
    vola = volatility(vola_series, vola_window) * 16
    median_volume = timeseries['Volume'].rolling(vola_window).median().iloc[-1]
    ma_fast = timeseries['Close'].rolling(ma_period_fast).mean().iloc[-1]
    ma_slow = timeseries['Close'].rolling(ma_period_slow).mean().iloc[-1]

#     ewma = timeseries['Close'].ewm(span=ewma_period).mean().iloc[-1]

    # Need the stocks to exist at least 3 years prior (756 trading days)
    if len(timeseries) < 756:
        eliminated_stocks = eliminated_stocks.append({'ticker': ticker,
                                                      'score': score,
                                                      'vola': vola,
                                                      'reason': 'umur belum 3 tahun'},
                                                     ignore_index=True)
        continue
    
    # If median volume falls below 100k in the stocks, drop it 
    if median_volume < 100000:
        eliminated_stocks = eliminated_stocks.append({'ticker': ticker,
                                                      'score': score,
                                                      'vola': vola,
                                                      'reason': 'avg volume di bawah 100k'},
                                                     ignore_index=True)
        continue
        
    # If it has been suspended (daily vol == 0) more than once, drop it
    if timeseries['Volume'].iloc[-momentum_window:].tolist().count(0) > 1:
        eliminated_stocks = eliminated_stocks.append({'ticker': ticker,
                                                      'score': score,
                                                      'vola': vola,
                                                      'reason': 'pernah disuspend lebih dari 1x'},
                                                     ignore_index=True)
        continue
    
    momentum_table = momentum_table.append({'ticker': ticker,
                                            'score': score,
                                            'vola': vola,
                                            'median_vol': median_volume,
                                            'ma_fast': ma_fast,
                                            'ma_slow': ma_slow},
                                           ignore_index=True)
    
    momentum_table['inv_vola'] = 1 / momentum_table['vola']

In [8]:
print(f'Ada {len(momentum_table)} saham lolos')
print(f'Ada {len(eliminated_stocks)} saham tereliminasi')

Ada 306 saham lolos
Ada 424 saham tereliminasi


In [9]:
momentum_table.sort_values('score', ascending=False)[:50].to_clipboard()

In [10]:
momentum_table.sort_values('score', ascending=False)[:50]

,ticker,score,vola,inv_vola,ma_fast,ma_slow,median_vol
165,LPLI,1822.438857,1.442909,0.693045,321.50000,182.632812,4.335000e+05
31,BAJA,1296.237873,0.956315,1.045681,333.75000,242.281250,3.749450e+06
224,PRIM,942.059515,1.624700,0.615498,374.03125,245.398438,6.067600e+06
254,SMDR,922.312641,0.815857,1.225705,578.75000,398.914062,8.182850e+06
57,BMSR,790.693921,0.900048,1.111052,200.71875,136.218750,1.689350e+06
277,TMAS,771.305722,1.295680,0.771795,293.00000,202.203125,8.939300e+06
176,MCAS,655.060702,0.550424,1.816781,7860.15625,5682.226562,4.077500e+05
191,MRAT,601.459737,0.771356,1.296418,381.06250,250.484375,1.469200e+06
116,HERO,545.809225,0.779800,1.282381,1567.50000,1141.523438,1.423500e+05
22,ARII,528.085230,0.995608,1.004411,297.18750,228.585938,4.030500e+05


In [13]:
eliminated_stocks[eliminated_stocks['reason'] == 'umur belum 3 tahun'].sort_values('score', ascending=False)

,ticker,score,vola,reason
40,BRIS,3815.691907,9.371545e-01,umur belum 3 tahun
82,DMMX,1200.297792,6.199791e-01,umur belum 3 tahun
44,CBMF,601.234993,3.262737e-01,umur belum 3 tahun
171,SAPX,529.212255,3.554760e-01,umur belum 3 tahun
43,INDO,466.787826,4.488161e-01,umur belum 3 tahun
...,...,...,...,...
83,KBAG,-85.764465,1.415350e+00,umur belum 3 tahun
0,ITIC,-90.342123,4.416746e-01,umur belum 3 tahun
172,HRME,-91.511054,5.381667e-09,umur belum 3 tahun
209,ARKA,-92.140816,5.981414e-01,umur belum 3 tahun


In [14]:
eliminated_stocks[eliminated_stocks['reason'] == 'pernah disuspend lebih dari 1x'].sort_values('score', ascending=False)

,ticker,score,vola,reason
163,TIFA,779.395081,0.305628,pernah disuspend lebih dari 1x
261,INCI,272.108060,0.410409,pernah disuspend lebih dari 1x
128,RANC,100.071218,0.348120,pernah disuspend lebih dari 1x
39,RMBA,74.438505,1.595118,pernah disuspend lebih dari 1x
307,RUIS,42.147570,0.923970,pernah disuspend lebih dari 1x
488,BIPP,39.918142,0.606751,pernah disuspend lebih dari 1x
109,WINS,35.294318,0.213005,pernah disuspend lebih dari 1x
60,LPPS,15.956782,0.699902,pernah disuspend lebih dari 1x
218,GWSA,13.444831,0.528288,pernah disuspend lebih dari 1x
34,AISA,8.955749,1.638821,pernah disuspend lebih dari 1x


In [15]:
eliminated_stocks[eliminated_stocks['reason'] == 'avg volume di bawah 100k'].sort_values('score', ascending=False)

,ticker,score,vola,reason
77,ARGO,489.785755,1.123822e+00,avg volume di bawah 100k
159,PTIS,358.443206,9.486231e-01,avg volume di bawah 100k
408,BBHI,323.138029,5.734052e-01,avg volume di bawah 100k
436,MERK,306.264092,4.505298e-01,avg volume di bawah 100k
358,PNSE,165.606997,3.973270e-07,avg volume di bawah 100k
...,...,...,...,...
70,PGLI,-66.517070,1.117693e+00,avg volume di bawah 100k
377,NASA,-68.554597,2.249972e-08,avg volume di bawah 100k
419,TARA,-71.126410,3.709058e-08,avg volume di bawah 100k
373,GLOB,-79.339016,5.216332e-01,avg volume di bawah 100k
